#### Importing packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error

#### getting original data

In [2]:
df = pd.read_csv("nifty_50_data.csv",index_col='Date', parse_dates=True)

In [4]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-01-03,4675.799805,4773.100098,4675.799805,4765.299805,4765.299805,0
2012-01-04,4774.950195,4782.850098,4728.850098,4749.649902,4749.649902,0
2012-01-05,4749.000000,4779.799805,4730.149902,4749.950195,4749.950195,0
2012-01-06,4724.149902,4794.899902,4686.850098,4754.100098,4754.100098,0
2012-01-09,4747.549805,4758.700195,4695.450195,4742.799805,4742.799805,0


In [6]:
def create_dataset(dataset, look_back):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

time_series_data = df['Close'].values.reshape(-1, 1)


In [8]:
# Function to perform LSTM prediction
def perform_LSTM(dataset, look_back, layer=5):
    dataset = dataset.astype('float32')
    dataset = np.reshape(dataset, (-1, 1))
    
    # # Normalize the data
    scaler = MinMaxScaler()
    dataset = scaler.fit_transform(dataset)
    
    # Split data into training and testing set
    train_size = int(len(dataset) * 0.9)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size, :], dataset[train_size:, :]
    
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    
    print(trainX.shape)
    print(trainY.shape)
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
    

    # Create and fit the LSTM network
    model = Sequential()
    model.add(LSTM(layer, input_shape=(1, look_back)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=adam)
    model.fit(trainX, trainY, epochs=100, batch_size=4, verbose=1)

    # Make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)

    # Invert predictions
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform([trainY])
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform([testY])
    testing_error = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))

    return testPredict, testY, testing_error


In [ ]:
look_back = 10

# Perform LSTM prediction
predicted_series, original_series,testing_error = perform_LSTM(time_series_data, look_back)

# Print the testing error
print("Testing RMSE:", testing_error)

# Plot original vs. predicted series
import matplotlib.pyplot as plt

plt.plot(original_series.flatten(), label='Original Series')
plt.plot(predicted_series.flatten(), label='Predicted Series')
plt.xlabel('Time')
plt.ylabel('Price')
plt.title('Original vs. Predicted Series')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Perform LSTM prediction
predicted_series, original_series, testing_error = perform_LSTM(time_series_data, look_back)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(original_series, predicted_series))

# Calculate MAE
mae = mean_absolute_error(original_series, predicted_series)

# Print RMSE and MAE
print("Testing RMSE:", rmse)
print("Testing MAE:", mae)

# Plot original vs. predicted series
plt.plot(original_series.flatten(), label='Original Series')
plt.plot(predicted_series.flatten(), label='Predicted Series')
plt.xlabel('Time')
plt.ylabel('Price')
plt.title('Original vs. Predicted Series')
plt.legend()
plt.show()


In [54]:
### model 2
def perform_LSTM(dataset, look_back, layer=10, optimizer='adam', learning_rate=0.001, batch_size=4):
    dataset = dataset.astype('float32')
    dataset = np.reshape(dataset, (-1, 1))
    
    # # Normalize the data
    # scaler = MinMaxScaler()
    # dataset = scaler.fit_transform(dataset)
    
    # Split data into training and testing set
    train_size = int(len(dataset) * 0.9)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size, :], dataset[train_size:, :]
    
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)

    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
    
    # Create and fit the LSTM network
    model = Sequential()
    model.add(LSTM(layer, input_shape=(1, look_back)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    model.fit(trainX, trainY, epochs=100, batch_size=batch_size, verbose=1)

    # Make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)

    # Invert predictions
    # trainPredict = scaler.inverse_transform(trainPredict)
    # trainY = scaler.inverse_transform([trainY])
    # testPredict = scaler.inverse_transform(testPredict)
    # testY = scaler.inverse_transform([testY])
    testing_error = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))

    return testPredict, testY, testing_error

In [ ]:
look_back = 10

# Perform LSTM prediction
predicted_series, original_series,testing_error = perform_LSTM(time_series_data, look_back)

# Print the testing error
print("Testing RMSE:", testing_error)

# Plot original vs. predicted series
# Set the background color
plt.figure(facecolor='lightgrey')

# Plot original and predicted series
plt.plot(original_series.flatten(), label='Original Series')
plt.plot(predicted_series.flatten(), label='Predicted Series')

# Set labels and title
plt.xlabel('Time')
plt.ylabel('Closing price')
plt.title('Original vs. Predicted Series')

# Add grid
plt.grid(True)

# Add legend
plt.legend()

# Show the plot
plt.show()

### Single layer LSTM Hyperparameter tuning

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam, Adadelta, Nadam
from sklearn.metrics import mean_squared_error
import numpy as np

# Check if GPU is available
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found. Please ensure TensorFlow is configured with GPU support.")

# Function to create dataset
def create_dataset(dataset, look_back):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

# Function to perform LSTM prediction
def perform_LSTM(dataset, look_back, neurons, optimizer, learning_rate, batch_size, num_executions=10):
    dataset = dataset.astype('float32')
    dataset = np.reshape(dataset, (-1, 1))

    # Normalize the data
    scaler = MinMaxScaler()
    dataset = scaler.fit_transform(dataset)

    # Split data into training and testing set
    train_size = int(len(dataset) * 0.8)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size, :], dataset[train_size:, :]

    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)

    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

    # Calculate the number of steps per epoch
    steps_per_epoch = len(trainX) // batch_size

    # Initialize lists to store RMSE scores
    rmse_scores = []

    # Loop over num_executions times
    for _ in range(num_executions):
        # Create and compile the LSTM model
        model = Sequential()
        model.add(LSTM(neurons, input_shape=(1, look_back)))
        model.add(Dense(1))
        model.compile(loss='mean_squared_error', optimizer=optimizer(learning_rate))

        # Fit the model
        model.fit(trainX, trainY, epochs=50, batch_size=batch_size, verbose=0, steps_per_epoch=steps_per_epoch)

        # Make predictions
        testPredict = model.predict(testX)

        # Invert predictions
        testPredict = scaler.inverse_transform(testPredict)
        testY_inv = scaler.inverse_transform([testY])

        # Calculate RMSE
        testing_error_rmse = np.sqrt(mean_squared_error(testY_inv[0], testPredict[:, 0]))
        rmse_scores.append(testing_error_rmse)

    # Calculate average RMSE score
    avg_rmse = np.mean(rmse_scores)

    return avg_rmse



#dataset and loookback

dataset = df['Close'].values.reshape(-1, 1)
look_back = 10


# Define the range of hyperparameters
neurons_list = [10, 30, 50, 100, 150, 200]
optimizer_list = [Adam, Adadelta, Nadam]
learning_rate_list = [0.1, 0.01, 0.001]
batch_size_list = [4, 8, 16]



# Perform grid search for hyperparameters
best_rmse = float('inf')
best_hyperparameters = None

for neurons in neurons_list:
    for optimizer in optimizer_list:
        for learning_rate in learning_rate_list:
            for batch_size in batch_size_list:
                avg_rmse = perform_LSTM(dataset, look_back, neurons, optimizer, learning_rate, batch_size)
                print(f"Neurons: {neurons}, Optimizer: {optimizer.__name__}, Learning Rate: {learning_rate}, Batch Size: {batch_size}, Average RMSE: {avg_rmse}")
                if avg_rmse < best_rmse:
                    best_rmse = avg_rmse
                    best_hyperparameters = (neurons, optimizer.__name__, learning_rate, batch_size)

print("Best Hyperparameters:")
print("Neurons:", best_hyperparameters[0])
print("Optimizer:", best_hyperparameters[1])
print("Learning Rate:", best_hyperparameters[2])
print("Batch Size:", best_hyperparameters[3])
